In [47]:
#import the library 
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium 
import requests
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans

In [2]:
# Get web content by request_get method
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#Prints the status of the request. If "200" the request was successful 
print(url.status_code)

200


In [3]:
#Creat a soup object
soup = BeautifulSoup(url.text,'html.parser')

In [4]:
#Find the Table in soup object 
table= soup.find('table')

In [5]:
#Creat a epity list "data" 
data=[]
#Interact for each row of table
for row in table.findAll('td'):
    #Creat a empit dictionary for each intraction
    cell = {}
    if row.span.text=='Not assigned':
    #if the field of the table contains  "Not assigned" pass to the next field
        pass 
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')'))\
                                 .replace(' /',',')).replace(')',' ')).strip(' ')
        data.append(cell)

In [6]:
#Creat a Data Frame with "data" 
df=pd.DataFrame(data)

#Replace neighborhood names with another, with proper accent and spacing
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

## Geting Latitude and longitude with GeoPy

In [7]:
#Creat "Latitude" and "Longitude" colouns in the "df" dataframe
df['Latitude']= None 
df['Longitude']= None 

I've tried using the geocorder but it doesn't respond so I'm using the Google API KEY for that.

In [8]:
#Set URL and KEY to do the search
google_url = 'https://maps.googleapis.com/maps/api/geocode/json?address='
API_KEY = 'A#############################################gvZ'

In [11]:
 for i in range(len(df)):
    # initialize your variable to None
    coords = None

    # loop until you get the coordinates
    while coords is None:
        response = requests.get('{}{}, Toronto, Ontario&key={}'\
                                .format(google_url,df['PostalCode'][i],API_KEY)).json()
        response.keys()
        if response['status'] == 'OK':
                    #Passing values latitude e longitude to dataframe df 
                    geometry = response['results'][0]['geometry']
                    df['Latitude'][i]=geometry['location']['lat']
                    df['Longitude'][i]=geometry['location']['lng']
                    coords = response['status']
   

## Clustering the neighborhoods in Toronto.



For this part we analisy the number "N" of neighborhood for each Borough and Clutering by the geografic codes and "N".

In [32]:
df['N_Neighborhood'] = df['Neighborhood'].apply(lambda x: len(x.split(',')))

In [ ]:
# set number of clusters
kclusters = 5

clustering = df.drop('PostalCode',1).drop('Borough',1).drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)

In [ ]:
# add clustering labels
df.insert(0, 'Cluster Labels', kmeans.labels_)

In [50]:
# create map
latitude = df.Latitude.mean()
longitude = df.Longitude.mean()

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Neighborhood'], df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters